# Async Caching in Python with aiocache and Redis

Caching is essential for building high-performance applications, especially when dealing with expensive computations or slow data sources. For modern Python async applications, [aiocache](https://aiocache.readthedocs.io/) is a popular library that provides a simple and flexible caching interface with support for multiple backends, including Redis.

## What is aiocache?

aioCache is an asynchronous caching library for Python, designed to work seamlessly with asyncio-based frameworks like FastAPI, Starlette, and aiohttp. It provides decorators and cache objects to easily cache function results, objects, or arbitrary data.

## Using aiocache with Redis Backend

### Installation

In [ ]:
# Install aiocache with Redis support
%pip install aiocache[redis]

# Install Redis CLI (choose your platform)

# For Ubuntu/Debian
# sudo apt-get install redis-tools

# For macOS using Homebrew
# brew install redis

# For Windows, download from Redis website
# https://github.com/microsoftarchive/redis/releases

# Key Features of aiocache

### **Async Support:** 

Designed for asyncio, works natively with async functions. Here's a basic example:

In [ ]:
from aiocache import SimpleMemoryCache
import asyncio


cache = SimpleMemoryCache()

# Using in-memory cache
async def cache_example():
    cache = SimpleMemoryCache()

    print("Setting cache values...")
    await cache.set("k1", "v1")
    await cache.set("k2", "v2")

    print("Getting cache values...")
    r1 = await cache.get("k1")
    print(f"Value for k1: {r1}")
    
    r2 = await cache.get("k2") 
    print(f"Value for k2: {r2}")

# Run the example
asyncio.run(cache_example())

### **Multiple Backends:** 

Supports Redis, Memcached, SimpleMemoryCache, and allows custom backend implementations.

In [8]:
from aiocache import RedisCache
import asyncio
import redis.asyncio as redis

# Using Redis cache with authentication
async def cache_example2():

    
    # Initialize Redis cache
    cache = RedisCache(
        namespace="main",
        endpoint="127.0.0.1", 
        port=6379,
        password="asdf"
    )

    print("Setting cache values...")
    await cache.set("k1", "v1")
    await cache.set("k2", "v2")

    print("Getting cache values...")
    r1 = await cache.get("k1")
    print(f"Value for k1: {r1}")
    
    r2 = await cache.get("k2") 
    print(f"Value for k2: {r2}")

# Run the example
await cache_example2()

Setting cache values...
Getting cache values...
Value for k1: v1
Value for k2: v2


### View Cache Values in Redis CLI

To inspect the cached values using Redis CLI:

In [40]:
import os
import subprocess

# Set Redis connection parameters
redis_url = "127.0.0.1"
redis_port = "6379"
password = "asdf"

def run_redis_cli(command):
    # Check if redis-cli is installed
    try:
        subprocess.run(["redis-cli", "--version"], check=True)
    except subprocess.CalledProcessError:
        print("redis-cli is not installed. Please install it first.")
        return

    # Connect to Redis using redis-cli
    cmd = f"redis-cli -h {redis_url} -p {redis_port} -a {password} {command}"
    
    try:
        # Run the redis-cli command and capture output
        result = subprocess.check_output(cmd, shell=True)
        
        # Handle empty results
        if not result.strip():
            result = "(empty)"

        result=  str(result)
        
        # Format multi-line output for better display
        if "\n" in result and len(result.strip().split("\n")) > 1:
            result = "\n" + result
        print("Redis connection successful.")
        print("result is ",result)
    except subprocess.CalledProcessError as e:
        print(f"Error connecting to Redis: {e}")



run_redis_cli("keys '*'")  


redis-cli 8.0.0
Redis connection successful.
result is  b'n1:get_user_data:123:detail=True\nmain:k2\nmain:k1\n'


We can see the keys created:

In [ ]:
main:k2
main:k1

### **Function Decorators:** 

`@cached` provides simple function caching with Redis. Here's an example:

In [13]:
from aiocache import cached, Cache
from aiocache.serializers import PickleSerializer

# Redis Cache with Decorator Example

# This example demonstrates using Redis as a caching layer with a decorator pattern. 
# The cache configuration uses aiocache library.

# ## Cache Configuration Details
# - **TTL (Time To Live)**: 10 seconds
# - **Cache Type**: Redis
# - **Serializer**: PickleSerializer for object serialization
# - **Namespace**: "n1" (used for key prefix)
# - **Connection**: localhost:6379
# - **Database**: 0
# - **Pool Size**: 10 concurrent connections max

# ## Cache Behavior
# - First call performs the actual operation and caches the result
# - Subsequent calls within TTL period retrieve data from cache
# - After TTL expiration, the cache is invalidated and the operation is performed again

# Using Redis cache with decorator
@cached(
    ttl=10,
    cache=Cache.REDIS, 
    serializer=PickleSerializer(),
    namespace="n1",
    endpoint="localhost",
    port=6379,
    password="asdf",
    db=0,
    pool_max_size=10
)
async def get_user_info(user_id: str) -> dict:
    # This would normally fetch from database
    # Simulating expensive operation
    await asyncio.sleep(1)  
    return {
        "id": user_id,
        "name": f"User {user_id}",
        "status": "active"
    }

# Usage example
async def cache_example3():
    # First call - cache miss
    result1 = await get_user_info("123")
    print("First call result:", result1)
    
    # Second call - cache hit (fetches from cache)
    result2 = await get_user_info("123")
    print("Second call result:", result2)

# Run the example
await cache_example3()

First call result: {'id': '123', 'name': 'User 123', 'status': 'active'}
Second call result: {'id': '123', 'name': 'User 123', 'status': 'active'}


We can see the key name in redis as:

In [26]:
run_redis_cli("keys '*'")

redis-cli 8.0.0
Redis connection successful.
result is  n1:get_user_data:123:detail=True
main:k2
main:k1



In [ ]:
n1:__main__get_user_info('123',)[]

## Custom Key Generation

You can customize how cache keys are generated using a key builder function. Here's an example:

In [48]:
from aiocache import cached, Cache
from typing import Any, Tuple, Dict

def custom_key_builder(
    func: Any,
    *args: Tuple[Any],
    **kwargs: Dict[str, Any]
) -> str:
    """Custom key builder for cache entries.
    
    Args:
        namespace: Cache namespace
        fn: The function being cached
        args: Positional arguments
        kwargs: Keyword arguments
    
    Returns:
        Formatted cache key as string
    """
    # Extract function name

    
    key = []
    key.append(func.__qualname__ or func.__name__)
    # Format args/kwargs into string
    args_str = ':'.join(str(arg) for arg in args)
    kwargs_str = ':'.join(f"{k}={v}" for k, v in kwargs.items())
    
    # Build key with components
    key_parts = key
    if args_str:
        key.append(args_str)
    if kwargs_str:
        key.append(kwargs_str)
        
    
    print("Key parts:", key,"LLL")
    return ":".join(key)

# Usage example
@cached(
    key_builder=custom_key_builder,
    ttl=1000,
    cache=Cache.REDIS, 
    serializer=PickleSerializer(),
    namespace="n1",
    endpoint="localhost",
    port=6379,
    password="asdf",
    db=0,
    pool_max_size=10
)
async def get_user_data(user_id: str, detail: bool = False):
    return {"id": user_id, "detail": detail}

# This will create keys like: "app:get_user_data:123:detail=True"

await get_user_data("123", detail=True)

Key parts: ['get_user_data', '123', 'detail=True'] LLL


{'id': '123', 'detail': True}

In [32]:
run_redis_cli("keys '*'")



redis-cli 8.0.0
Redis connection successful.
result is  n1:get_user_data:123:detail=True
main:k2
main:k1



The key name in the cache can be observed as:

In [ ]:
n1:get_user_data:123:detail=True

In [41]:
run_redis_cli("GET n1:get_user_data:123:detail=True")


redis-cli 8.0.0
Redis connection successful.
result is  b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x02id\x94\x8c\x03123\x94\x8c\x06detail\x94\x88u.\n'


The value is:

In [ ]:
# 127.0.0.1:6379> GET n1:get_user_data:123
# "\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x02id\x94\x8c\x03123\x94\x8c\x06detail\x94\x89u."

This gives you complete control over the cache key format while maintaining a clear namespace structure.

**Cache Invalidation:** Invalidate cache by key, pattern, or globally.

### Cache Invalidation

In [42]:
async def cache_example5():

    cache = RedisCache( namespace="main",endpoint="127.0.0.1", port=6379,password="asdf")

    await cache.delete("n1:get_user_data:123") # Delete specific cache entry

    await cache.clear(namespace="n1")  # Clear all cache entries with namespace "n1"

    await cache.clear()  # Clear all cache entries

await cache_example5()

**Serialization:**

**Choosing a Serializer:**

Each serializer has specific use cases:

- **PickleSerializer (Default):**
    - Fastest performance
    - Handles complex Python objects
    - Security risk: vulnerable to code execution
    - Best for trusted internal systems

- **JsonSerializer:**
    - Human readable
    - Language-agnostic
    - Limited to JSON-serializable types
    - Best for web APIs or cross-service caching

- **StringSerializer:**
    - Minimal overhead
    - Limited to string data
    - Best for simple string caching

Example:

In [57]:
from aiocache.serializers import JsonSerializer, PickleSerializer


from aiocache import cached, Cache
from typing import Any, Tuple, Dict

from aiocache import cached, Cache
from typing import Any, Tuple, Dict

def custom_key_builder(
    func: Any,
    *args: Tuple[Any],
    **kwargs: Dict[str, Any]
) -> str:
    """Custom key builder for cache entries.
    
    Args:
        namespace: Cache namespace
        fn: The function being cached
        args: Positional arguments
        kwargs: Keyword arguments
    
    Returns:
        Formatted cache key as string
    """
    # Extract function name

    
    key = []
    key.append(func.__qualname__ or func.__name__)
    # Format args/kwargs into string
    args_str = ':'.join(str(arg) for arg in args)
    kwargs_str = ':'.join(f"{k}={v}" for k, v in kwargs.items())
    
    # Build key with components
    
    if args_str:
        key.append(args_str)
    if kwargs_str:
        key.append(kwargs_str)
        
    
    print("Key parts:", key,"LLL")
    return ":".join(key)

# Usage example
@cached(
    key_builder=custom_key_builder,
    ttl=1000,
    cache=Cache.REDIS, 
    serializer=JsonSerializer(),
    namespace="n2",
    endpoint="localhost",
    port=6379,
    password="asdf",
    db=0,
    pool_max_size=10
)
async def get_user_data(user_id: str, detail: bool = False):
    return {"id": user_id, "detail": detail}

await get_user_data("123")

Key parts: ['get_user_data', '123'] LLL


{'id': '123', 'detail': False}

In [58]:
run_redis_cli("keys '*'")

redis-cli 8.0.0
Redis connection successful.
result is  b'n1:get_user_data:123\nn1:get_user_data:123:detail=True\nn2:get_user_data:123\n'


We can see that with json serializer the output is stored in json format in redis:

In [61]:
run_redis_cli("GET n2:get_user_data:123")

redis-cli 8.0.0
Redis connection successful.
result is  b'{"id": "123", "detail": false}\n'


In [ ]:
127.0.0.1:6379> GET n1:example6:1234
"{\"id\": \"1234\", \"detail\": false}"

## Conclusion

aioCache makes it easy to add async caching to your Python applications, with powerful features and Redis support out of the box. It is a great choice for modern web APIs and microservices that need scalable, non-blocking caching.

For more details, visit the [aiocache documentation](https://aiocache.readthedocs.io/).

The example code for this guide can be found at:
- GitHub repository: [neural-engineer/example-projects](https://github.com/neural-engineer/example-projects)
- File path: `/redis-persistance/redis-cache.md`